In [2]:
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.optimization.algorithms import MinimumEigenOptimizer, CplexOptimizer
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToQubo
from qiskit.optimization.algorithms.optimization_algorithm import OptimizationResultStatus
import numpy as np
from helpers import *
DIR = 'TEST_DATA/'
Q_SEED = 10598 # as used in most issues
aqua_globals.random_seed = Q_SEED
shots = 4096

In [3]:
# select linear program to solve
qps = createModelsFromDir(DIR)
qp = qps['test_6']

In [4]:
# init classical Optimizers
optimizer = COBYLA() # SLSQP is default in Class VQE
cplex = CplexOptimizer()

In [5]:
# solve qps with Minimum Eigen Optimizer QAOA
backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend,
                                   seed_simulator=Q_SEED,
                                   seed_transpiler=Q_SEED,
                                   shots=shots)  # set parameters like noise model, error mitigation and coupling map...

# QAOA parameters:

p = 2 # circuit depth grows linear with p x qp.get_num_linear_constraints()
initial_point = np.zeros(2 * p)  # default in this setting... Issue : QAOA no longer generates random initial points #1417
qaoa_mes = QAOA(p = p, quantum_instance=quantum_instance, optimizer = optimizer, initial_point = initial_point)


qaoa = MinimumEigenOptimizer(qaoa_mes)

In [6]:
cplex.solve(qp)

optimal function value: -3.0
optimal value: [1. 0. 0. 0. 1. 0.]
status: SUCCESS

In [7]:
print("number of qubits: ",qp.qubo_model.get_num_vars())

number of qubits:  19


In [8]:
res = qaoa.solve(qp)
res

optimal function value: 0.0
optimal value: [1. 1. 1. 0. 1. 1.]
status: SUCCESS

In [12]:
qp.is_feasible(res.x)

True